# Il verme è tratto
Cercherò di creare una trasposizione di questo gioco da tavola, se mai ci riesca.

## Idee
Quello che potrei sviluppare

* Per iniziare potrei gestire il gioco fra persone
* Poi potrei integrare un pochetto di grafica (guizero?)
* Poi potrei inserire intelligenza artificiale (difficile, basata sulle probabilità)
* Infine potrei tentare di integrare il gioco in rete

## Regolamento
Documento contenente le regole

In [92]:
%%HTML
<iframe src=./regole.pdf width=100% height=350></iframe>

## Svolgimento
In questa sezione cercherò di spiegare i passaggi della partita per comprendere meglio l'algoritmo da adottare

<span class="mark">1. Si decide il numero di giocatori (da 2 a 7) e li si ordina per età dal più giovane al più vecchio (in origine si gioca in senso orario a partire dal più giovane)</span>
<span class="mark">2. Si dispongono le tessere sul tavolo a formare una griglia in ordine numerico (da 21 a 36)</span>
3. Il primo giocatore lancia i dadi:
    1. lancia 8 dadi e sceglie quale serie tenere con lo stesso numero in comune e può rilanciare i dadi rimanenti fino a quando:
        * ottiene un numero maggiore o uguale a una tessera disponibie dalla griglia del tavolo
        * ottiene un numero uguale ad una tessera in cima ad una pila di un altro giocatore
        * non ci sono più serie con numeri in comune non precedentemente trattenuti (non si possono scegliere piu volte serie con lo stesso numero in comune)
        * non si è almeno trattenuto una volta una serie con il simbolo del verme
        * non ci sono più dadi da rilanciare
    3. il giocatore prende una una tessera e la mette in cima alla sua pila se:
        * ha totalizzato un numero maggiore o uguale ad una delle tessere disposte sulla griglia, prende una di queste
        * ha totalizzato un numero uguale a una delle tessere in cima alla pila di un altro giocatore, prende questa
    4. in tutti gli altri casi ha sballato e quindi, se possiede delle tessere, dovrà:
        * restituire quella in cima alla sua pila reinserendola nella griglia
        * se nella griglia ci sono ancora tessere dal valore più alto di quella restituita bisognerà capovolgere (e quindi rendere indisponibile) la tessera dal valore più alto presente nella griglia
4. Si passa al giocatore seguente
5. Il gioco ha fine quando non ci sono più tessere sottraibili dalla griglia
6. Vince il giocatore che ha totalizzato più vermi (ogni tessera oltre al valore numerico ha anche un punteggio rappresentato da dei vermi, da uno a quattro)

## Codice

In [93]:
import doctest, random, itertools, os

In [94]:
class Interfaccia:
    """
    Questa classe deve contenere le azioni eseguite in base al tipo
    di interfaccia implementata
    
    >>> i = Interfaccia()
    >>> i.inizializza_dadi()
    (Dado(6), Dado(6), Dado(6), Dado(6), Dado(6), Dado(6), Dado(6), Dado(6))
    >>> len(i.inizializza_dadi())
    8
    """
    
    
    def inizializza_dadi(self):
        """Crea una tupla con tutti i dadi in gioco"""
        return tuple(Dado() for i in range(8))
    
    def inizializza_tessere(self):
        """
        metodo che restituisce le tessere ordinate per la grigla sul tavolo 
        può richiedere la scelta delle immagini?
        """
        tessere = []
        p = 1
        for v in range (21, 37):
            tessere.append(Tessera(v, int(p)))
            p += 0.25
        return tessere  
    def inizializza_giocatori(self):
        """metodo che richiede i giocatori e ritorna una loro lista"""
        raise NotImplementedError
    def scelta_dadi(self):
        """metodo che gestisce i lanci di un giocatore"""
        raise NotImplementedError     

In [95]:
class Tui(Interfaccia):
    def inizializza_giocatori(self):
        """richiede partecipanti, restituisce lista di giocatori ordinati per età"""
        gioca_num = int(input("inserisci numero giocatori(2-7): "))
        gioca_num = min(max(2, gioca_num), 7)   #no minore di 2, non maggioree di 7
        nomi = []
        giocatori = []
        while len(giocatori) < gioca_num:
            nome = input("inserisci nome giocatore: ")
            if nome in nomi:
                print("Giocatore già esistente, usare un altro nome")
                continue
            nomi.append(nome)
            while True:
                eta  = input("inserisci la sua età o niente per random: ")
                try:
                    eta = int(eta)
                except ValueError:
                    if eta == "":                     #se non si specifica l'età
                        eta = random.randint(0,100)   #viene estratta random
                        print("Età random: {} anni".format(eta))
                finally:
                    if isinstance(eta, int): break
                    else: print("età sbagliata, ritenta")
            giocatori.append(Giocatore(nome, eta))
        giocatori.sort(key=lambda giocatore:giocatore.eta)
        return giocatori

In [102]:
class Gui(Interfaccia):
    """
        Esempio di come potrebbe funzionare gui, ad esempio ho reimplementato i metodi
        per le tessere e i dadi così da aggiungergli le immagini
        
        Aggiunge le immagini ai dadi
        >>> g = Gui()
        >>> d = g.inizializza_dadi()
        >>> fd = ('dado_gif/1.gif', 'dado_gif/2.gif', 'dado_gif/3.gif', 'dado_gif/4.gif', 'dado_gif/5.gif', 'dado_gif/6.gif')
        >>> all(dado.facced == fd for dado in d)
        True
        >>> from os.path import isfile
        >>> all([isfile(dado) for dado in fd])
        True
        
        Riesce ad utilizzare il metodo della classe padre per generare la lista tessere
        >>> g = Gui()
        >>> t = g.inizializza_tessere()
        >>> t
        [Tessera(21, 1), Tessera(22, 1), Tessera(23, 1), Tessera(24, 1), Tessera(25, 2), Tessera(26, 2), Tessera(27, 2), Tessera(28, 2), Tessera(29, 3), Tessera(30, 3), Tessera(31, 3), Tessera(32, 3), Tessera(33, 4), Tessera(34, 4), Tessera(35, 4), Tessera(36, 4)]
    
        E inoltre gli aggiunge dei path validi per le immagini
        >>> from os.path import isfile
        >>> all([isfile(tes.immagine) for tes in t])
        True
    """
    def inizializza_dadi(self):
        dadi = super().inizializza_dadi()
        facced = tuple(os.path.join("dado_gif/", str(f) + ".gif") for f in dadi[0].faccev)
        for dado in dadi:
            dado.facced = facced
        return dadi
            
    
    def inizializza_tessere(self):
        tessere = super().inizializza_tessere()
        for tessera in tessere:
            tessera.immagine = os.path.join("tessere_gif/", str(tessera.valore) + ".gif")
        return tessere

In [97]:
class Giocatore():
    """
    Questa classe rappresenta un giocatore
    
    Istanziando il giocatore bisogna specificare il nome e l'età (facoltativa)
    >>> g = Giocatore('aldo', 14)
    >>> g.nome  #il nome del giocatore
    'aldo'
    >>> g.eta   #la sua età
    14
    
    Senza età viene assegnata un età casuale da 0 a 100
    >>> g = Giocatore('rino')
    >>> isinstance(g.eta, int) and 0 <= g.eta <= 100
    True
    
    Età non valida: ValueError
    >>> g = Giocatore('rinco', 'asda')
    Traceback (most recent call last):
    ...
    ValueError: invalid literal for int() with base 10: 'asda'
    
    Età valida
    >>> g = Giocatore('rinco', '134')
    >>> g.eta
    134
    """
    def __init__(self, nome, eta=None):
        self.nome = nome
        if eta is None: self.eta = random.randint(0,100) #età casuale
        else: self.eta = int(eta)
            
    def __repr__(self):
        return 'Giocatore("{}", {})'.format(self.nome, self.eta)

In [98]:
class Tessera():
    """
    Questa classe rappresenta una tessera
    
    >>> t = Tessera(21, 1)
    >>> t
    Tessera(21, 1)
    """
    def __init__(self, valore, punti):
        self.valore = int(valore)
        self.punti = int(punti)
    
    def __repr__(self):
        return "Tessera({}, {})".format(self.valore, self.punti)

In [99]:
class Dado():
    """
    Questa classe rappresenta un dado
    
    >>> d = Dado(6)
    >>> d.faccev
    (1, 2, 3, 4, 5, 6)
    
    >>> d.lancia() in [1,2,3,4,5,6]
    True
    """
    def __init__(self, facce=6):
        self.faccev = tuple(range(1, facce+1))
    
    def __repr__(self):
        return "Dado({})".format(len(self.faccev))
        
    # def lancia(self, numero=1):
    #     """Lancia il dado n volte"""
    #     return [random.choice(self.faccev) for n in range(numero)]  
    
    def lancia(self):
        """lancia il dado una sola volta"""
        return random.choice(self.faccev)

In [110]:
class Tavolo():
    """
    Questa classe serve a gestire gli spostamenti sul tavolo
    
    disattivo questo test perché è interattivo, comunuqe inserendo come input
    2, a, 2, b, 3, funziona correttamente
    >> tui = Tui()
    >> tav = Tavolo(tui)
    >> isinstance(tav.giocatori[0], Giocatore)
    True
    >> isinstance(tav.griglia[0], Tessera)
    True
    """
    def __init__(self, interf=Interfaccia()):
        self.giocatori = interf.inizializza_giocatori()
        self.griglia = interf.inizializza_tessere()
        #self.griglia = self.inizializza_griglia_tessere()

    
    def inizializza_griglia_tessere(self):
        """Questa funzione genera una lista di tessere"""
        tessere = []
        p = 1
        for v in range (21, 37):
            tessere.append(Tessera(v, int(p)))
            p += 0.25
        return tessere

### doctest

In [111]:
doctest.testmod()

inserisci numero giocatori(2-7): 1
inserisci nome giocatore: a
inserisci la sua età o niente per random: 2
inserisci nome giocatore: b
inserisci la sua età o niente per random: 3


TestResults(failed=0, attempted=31)

### codice inutilizzato

In [ ]:
def inizializza_griglia_tessere(car="tessere_gif/", est=".gif"):
    """
    Questa funzione genera una lista di tessere
    
    >>> ts = genera_tessere()
    >>> ts
    [(21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 2), (27, 2), (28, 2), (29, 3), (30, 3), (31, 3), (32, 3), (33, 4), (34, 4), (35, 4), (36, 4)]
    >>> from os.path import isfile
    >>> all([isfile(t.immagine) for t in ts])
    True
    """
    tessere = []
    p = 1
    for v in range (21, 37):
        path = car + str(v) + est
        tessere.append(Tessera(v, int(p), path))
        p += 0.25
    return tessere

In [ ]:
#metto i doctest dentro un altra funzione alla fine, se non li voglio nelle docstring
#di ogni singola classe e funzione potrei fare così e metterli a parte
def test_giocatore():
    """
    Questa classe rappresenta un giocatore
    
    Istanziando il giocatore bisogna specificare il nome e l'età (facoltativa)
    >>> g = Giocatore('aldo', 14)
    >>> g.nome  #il nome del giocatore
    'aldo'
    >>> g.eta   #la sua età
    14
    
    Senza età viene assegnata un età casuale da 0 a 100
    >>> g = Giocatore('rino')
    >>> isinstance(g.eta, int) and 0 <= g.eta <= 100
    True
    
    Età non valida sempre età casuale
    >>> g = Giocatore('rinco', 'asda')
    >>> isinstance(g.eta, int) and 0 <= g.eta <= 100
    True
    
    Età valida
    >>> g = Giocatore('rinco', '134')
    >>> g.eta
    134
    """
    pass
doctest.testmod()